<a href="https://colab.research.google.com/github/hasune613/hello-world/blob/main/XGBoost_%2B_KFold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q semopy

     |████████████████████████████████| 1.6 MB 5.4 MB/s 


In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler,StandardScaler,RobustScaler
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

from warnings import filterwarnings
filterwarnings('ignore')

import os

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/kaggle/Tabular Playground Series sep_2021/train.csv')
test = pd.read_csv('/content/drive/MyDrive/kaggle/Tabular Playground Series sep_2021/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/kaggle/Tabular Playground Series sep_2021/sample_solution.csv')


In [ ]:
# train.head(50)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 957919 entries, 0 to 957918
Columns: 120 entries, id to claim
dtypes: float64(118), int64(2)
memory usage: 877.0 MB


In [ ]:
train.describe()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,...,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,claim
count,957919.000000,942672.000000,942729.000000,942428.000000,942359.000000,942514.000000,942398.000000,942415.000000,9.425460e+05,9.426700e+05,942696.000000,942494.000000,9.423260e+05,942455.000000,942697.000000,942410.000000,942475.000000,942492.000000,942594.000000,942445.000000,942464.000000,942465.000000,942641.000000,942563.000000,942289.000000,942413.000000,9.425610e+05,9.424750e+05,9.426540e+05,942504.000000,942527.000000,942241.000000,9.423900e+05,9.424270e+05,942671.000000,9.425850e+05,942556.000000,942607.000000,942485.000000,942360.000000,...,942599.000000,942573.000000,9.424340e+05,942292.000000,9.425340e+05,942470.000000,9.423960e+05,942603.000000,942371.000000,942474.000000,942437.000000,942412.000000,942427.000000,942462.000000,942505.000000,942320.000000,942634.000000,942654.000000,9.426310e+05,942485.000000,942393.000000,942570.000000,9.427510e+05,942319.000000,942721.000000,942533.000000,942375.000000,942535.000000,9.425790e+05,942390.000000,942554.000000,942420.000000,942509.000000,942686.000000,942481.000000,942360.000000,9.423300e+05,942512.000000,942707.000000,957919.000000
mean,478959.000000,0.090201,0.345964,4068.744207,0.201214,0.304869,-0.071458,1620.843815,3.771642e+05,1.806054e+15,5323.442367,0.729518,1.844436e+09,0.247759,6.999152,0.019394,444.804439,6.892520,4.491982,22.448660,203.782031,61070.567499,2.269597,87.166891,0.341240,414.983986,3.379988e+12,1.254164e+12,2.256887e+06,0.329054,7.882455,0.394211,1.344700e+05,3.579055e+05,-0.000005,2.782318e+16,185.485192,406.072571,1.768743,1981.074344,...,0.194237,3.240370,1.053811e+11,152.850973,6.128694e+06,635.301741,3.251414e+10,26.598567,207.260994,3805.100719,6.733556,0.366754,4868.608259,132.264261,0.821190,13.120677,3849.687894,0.999970,1.415684e+13,1.683151,0.425840,20.213776,3.215760e+05,548.647493,3855.616303,0.178027,0.160832,0.014196,1.671873e+09,0.398622,-19.926398,2.074530,23.885245,1.748777,63152.973540,1.208876,4.276905e+16,3959.204669,0.559267,0.498492
std,276527.540591,0.043564,0.146251,6415.829440,0.212510,0.145343,2.123777,1276.281403,3.454325e+05,2.335204e+15,10068.380032,1.495535,2.125478e+09,0.101156,6.621472,0.101880,631.327086,1.715735,3.899241,14.609968,281.015052,89859.616863,0.896001,37.350264,1.643595,314.822410,5.655524e+12,1.642736e+12,2.303909e+06,0.433861,5.939864,0.781932,2.036605e+05,4.626670e+05,0.001534,3.452707e+16,701.769083,314.708639,0.588984,1958.688237,...,0.162364,1.992068,9.896094e+10,227.888751,8.767985e+06,583.468704,3.069150e+10,25.454605,158.215276,3532.626437,3.158678,0.146413,8428.424099,110.058170,0.712584,12.740238,6437.386650,0.001535,1.640385e+13,0.712016,0.283651,19.617660,3.277370e+05,864.175628,6670.256245,0.123319,0.141616,0.016259,1.875733e+09,0.298071,18.578439,0.895793,45.581360,10.088848,92435.016241,0.114959,6.732441e+16,3155.991777,0.408426,0.499998
min,0.000000,-0.149910,-0.019044,-9421.700000,-0.082122,-0.006990,-12.791000,-224.800000,-2.984300e+04,-1.153300e+15,-26404.000000,-8.086300,-4.081000e+08,-0.103800,-0.853760,-0.335660,-116.880000,-3.664500,-0.066527,-4.422500,-58.834000,-84079.000000,-6.009400,-20.514000,-5.735200,-71.502000,-6.956700e+11,-9.384200e+11,-4.706000e+05,-0.005659,-0.529990,-3.813500,-3.496500e+05,-6.055900e+05,-0.003881,-2.068900e+16,-2414.300000,-40.881000,0.546100,-433.700000,...,-0.017615,0.964200,-7.345700e+10,-28.752000,-2.992000e+06,-74.545000,-5.949500e+09,-7.616400,-22.576000,-296.780000,-0.257570,-0.012238,-12829.000000,-12.922000,-3.293300,-1.352400,-7764.300000,0.996100,-5.714600e+12,0.608200,-0.034559,-4.294900,-2.277700e+05,-222.210000,-11581.000000,-0.029027,-0.066726,-0.007535,-5.877000e+08,-0.042355,-105.860000,0.27704

In [ ]:
train.isnull().sum()

id           0
f1       15247
f2       15190
f3       15491
f4       15560
         ...  
f115     15559
f116     15589
f117     15407
f118     15212
claim        0
Length: 120, dtype: int64

In [ ]:
test.head(3)

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,...,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118
0,957919,0.16585,0.48705,1295.00,0.02310,0.31900,0.90188,573.29,3743.7,2.705700e+12,6221.00,1.17200,5.175000e+09,0.25831,3.42050,0.044983,1001.600,10.1040,10.5360,19.517,-1.1209,1577.6,2.1847,96.109,1.1535,95.865,2.847100e+12,9.642100e+10,6406000.0,0.00299,6.6353,-0.015913,21723.0,67604.0,-0.000227,6.046200e+16,203.580,301.790,1.3461,5524.6,...,NaN,0.510410,4.0322,2.477900e+11,69.264,5494800.0,1239.00,5.349300e+08,5.3086,182.59,829.17,5.2319,0.010301,3640.80,112.12,-0.43442,11.16100,3755.7,0.99926,5.338200e+10,1.3867,-0.001878,58.7940,75657.0,187.24,1258.90,0.16334,0.055398,0.020002,795640000.0,0.16253,-22.1890,2.0655,0.43088,-10.7410,81606.0,1.1940,1.980400e+14,2017.1,0.46357
1,957920,0.12965,0.37348,1763.00,0.72884,0.33247,-1.26310,875.55,554370.0,5.955700e+14,934.43,1.89790,4.905100e+09,0.28471,2.13870,-0.034328,35.223,6.5873,12.7340,29.590,4.1639,31915.0,3.1158,109.430,-0.8522,991.370,4.700100e+09,1.618200e+12,967810.0,1.02150,2.9617,0.810310,415570.0,384720.0,0.002243,6.712900e+15,228.590,33.952,1.4938,2067.5,...,-5.3544,0.067286,5.4044,2.462600e+11,810.300,860670.0,2165.20,2.711500e+08,10.2320,515.41,262.71,5.6357,0.374220,420.16,116.23,1.26980,0.78619,1207.0,1.00070,2.065000e+13,1.2888,0.248850,9.9857,16323.0,244.46,647.58,0.25382,0.008800,0.018479,112060000.0,0.81528,-1.6342,1.5736,-1.07120,11.8320,90114.0,1.1507,4.388000e+16,6638.9,0.28125
2,957921,0.12019,0.44521,736.26,0.04615,0.29605,0.31665,2659.50,317140.0,3.977800e+14,131.81,-0.48404,3.800000e+06,0.37173,0.40245,0.015119,1991.600,6.0050,0.9919,11.949,888.0700,887.7,3.5237,108.140,-1.6542,528.140,2.463600e+12,9.005900e+09,7259.3,1.00740,2.6123,1.062500,-208010.0,65708.0,0.001391,8.129800e+13,-84.032,NaN,1.7556,4729.2,...,-1.4805,0.064203,3.0197,1.684400e+11,212.830,7891000.0,277.16,4.047900e+10,1.1170,393.51,8859.10,8.8098,0.405350,393.65,305.14,1.32660,30.85200,2803.7,0.99925,4.722600e+12,1.5003,0.493380,37.0470,139070.0,2464.00,4724.80,0.29916,0.093046,0.018516,65193000.0,0.81831,-32.7800,2.1364,-1.93120,-3.2804,37739.0,1.1548,1.718100e+14,5844.0,0.13797


In [ ]:
test.isnull().sum()

id         0
f1      7812
f2      7891
f3      7795
f4      7733
        ... 
f114    7942
f115    7977
f116    8083
f117    7763
f118    7885
Length: 119, dtype: int64

In [ ]:
submission.head(3)

,id,claim
0,957919,0.5
1,957920,0.5
2,957921,0.5


In [ ]:
train.shape,test.shape,submission.shape

((957919, 120), (493474, 119), (493474, 2))

In [ ]:
train.columns

Index(['id', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9',
       ...
       'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116', 'f117', 'f118',
       'claim'],
      dtype='object', length=120)

In [ ]:
test.columns

Index(['id', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9',
       ...
       'f109', 'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116', 'f117',
       'f118'],
      dtype='object', length=119)

In [ ]:
submission.columns

Index(['id', 'claim'], dtype='object')


### Feature Engineering

In [ ]:
features =[x for x in train.columns.values if x[0]=='f']


In [ ]:
train['n_missing'] = train[features].isna().sum(axis=1)
test['n_missing'] = test[features].isna().sum(axis=1)

train['abs_sum'] = train[features].abs().sum(axis = 1)
test['abs_sum'] = test[features].abs().sum(axis=1)

train['sem'] = train[features].sem(axis=1)
test['sem'] = test[features].sem(axis=1)

train['std'] = train[features].std(axis = 1)
test['std'] = test[features].std(axis = 1)

train['avg'] = train[features].mean(axis = 1)
test['avg'] = test[features].mean(axis = 1)

train['max'] = train[features].max(axis= 1)
test['max'] = test[features].max(axis = 1)

train['min'] = train[features].min(axis= 1)
test['min'] = train[features].min(axis = 1)

In [ ]:
train[['abs_sum']]

,abs_sum
0,1.294749e+16
1,4.579784e+16
2,4.439307e+14
3,2.125939e+15
4,1.264657e+16
...,...
957914,5.616945e+16
957915,1.831924e+17
957916,2.219898e+16
957917,7.790588e+16


In [ ]:
train[['n_missing']]

,n_missing
0,1
1,0
2,5
3,2
4,8
...,...
957914,0
957915,4
957916,0
957917,1


In [ ]:
imputer = SimpleImputer(strategy = 'median')
for col in features:
    train[col] = imputer.fit_transform(np.array(train[col]).reshape(-1,1))
    test[col] = imputer.fit_transform(np.array(test[col]).reshape(-1,1))


In [ ]:
r_scaler = RobustScaler()
for col in features:
    train[col] = r_scaler.fit_transform(np.array(train[col]).reshape(-1,1))
    test[col] = r_scaler.fit_transform(np.array(test[col]).reshape(-1,1))



In [ ]:
X = train.drop(['id', 'claim'],axis =1)
y = train['claim']

In [ ]:
X.shape, y.shape

((957919, 125), (957919,))

In [ ]:
y.head(10)

0    1
1    0
2    1
3    1
4    1
5    0
6    0
7    1
8    0
9    0
Name: claim, dtype: int64

In [ ]:
# XGBClassifier().get_params()

xgb_params = {
    'objective': 'binary:logistic',
    'eval_metric':'auc',
    'n_estimators': 100,
    'max_depth': 3,
    'gamma': 0.2465,
    'subsample': 0.6423,
    'colsample_bytree': 0.775,
    'colsample_bylevel': 0.868,
    'min_child_weight': 366,
    'reg_lambda': 0.05,
    'reg_alpha': 10,
    'verbosity': 1,
    'random_state': 42,
}

In [ ]:
model = XGBClassifier(**xgb_params)
model


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=0.868,
              colsample_bynode=1, colsample_bytree=0.775, eval_metric='auc',
              gamma=0.2465, learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=366, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=42,
              reg_alpha=10, reg_lambda=0.05, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.6423, verbosity=1)

In [ ]:
X_test = test.drop(['id'],axis =1)
X_test.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,...,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,n_missing,abs_sum,sem,std,avg,max,min
0,1.682833,0.573550,0.004415,-0.443982,-0.053354,0.636551,-0.439935,-0.621453,-0.165071,0.937569,-0.005963,1.261560,-0.206224,-0.204495,0.345432,1.369303,2.136912,1.224448,-0.015971,-0.153971,-0.222443,-0.125338,-0.011153,0.030756,-0.510253,0.533258,-0.106840,1.278949,-0.084207,-0.026809,-0.709438,-0.180168,-0.220448,-0.094665,1.037804,0.501464,-0.105060,-0.371726,1.660777,-0.099207,...,-0.507804,-0.424231,0.008473,-0.446564,-0.278695,-2.901656,0.472210,0.037276,-2.080010,0.105921,0.692302,-0.307018,-0.281347,-0.029868,-1.108238,1.718278,-0.326304,0.146823,0.069742,0.029222,-0.418132,0.569546,-0.002322,-0.392036,-0.332944,0.406834,-0.068584,-1.050090,0.744249,0.180451,-0.254090,-0.255872,-0.022298,1,6.096641e+16,5.167411e+14,5.589410e+15,5.210804e+14,6.046200e+16,-3.756600e+01
1,0.877852,-0.090379,0.125237,2.298564,0.026322,-0.438367,-0.286828,0.575653,0.029832,-0.026679,0.603704,1.181715,0.100325,-0.345209,-0.453232,-0.081401,-0.132073,1.634026,0.476018,-0.136364,0.165072,1.019926,0.373728,-0.970650,1.151537,-0.206409,0.628486,-0.153275,1.051780,-0.503040,0.161735,2.318021,0.449896,0.983851,-0.061876,0.581042,-0.601776,-0.106746,0.267757,-0.098297,...,-0.513168,-0.273950,1.278687,-0.571500,-0.166506,-0.258624,-0.123839,0.062309,0.317843,-0.482264,0.050781,0.324561,0.535832,-0.222244,-0.396012,-0.170303,-0.464990,0.240167,-0.084124,0.450909,-0.701696,0.498908,-0.238172,0.917392,0.565992,-0.090486,-0.147859,0.984214,0.846088,-0.284640,0.592787,0.713465,-0.420090,0,5.133270e+16,3.756839e+14,4.080971e+15,4.350229e+14,4.388000e+16,-5.675800e+01
2,0.667489,0.328954,-0.139834,-0.354408,-0.189104,0.345983,0.616814,0.059896,-0.035191,-0.173073,-1.396834,-0.268238,1.110776,-0.535811,0.044701,2.855470,-0.507775,-0.554011,-0.385611,2.808455,-0.231256,1.521648,0.336456,-1.371072,0.291920,0.433461,-0.149079,-0.406250,1.036054,-0.548335,0.427645,-1.637374,-0.224455,0.611640,-0.197555,-0.413682,0.000000,0.362935,1.340275,-0.091784,...,0.304486,-0.552174,0.813453,1.324477,0.715366,-0.032537,-0.128745,1.212922,0.397761,1.222273,0.452679,-0.311404,-0.096094,0.193358,0.298607,0.876803,-0.178084,3.860938,0.942095,0.662218,-0.189033,0.500624,-0.254342,0.923470,-0.796128,0.478516,-0.193247,-0.377732,0.219168,-0.240602,-0.254598,0.546749,-0.732704,1,6.602766e+14,3.748218e+12,4.054317e+13,5.633213e+12,3.977800e+14,-4.453500e+13
3,-0.804203,0.041039,-0.072741,2.805694,0.229268,0.104062,-0.534521,0.078463,-0.177244,-0.201944,-2.778650,1.113763,-0.110079,-0.459587,-0.949111,2.849315,0.068972,1.333830,0.162157,0.294765,-0.253009,0.544772,-0.418483,1.370773,-0.122959,-0.211056,-0.157453,0.196970,0.023459,-0.248538,0.003142,-0.008588,0.058491,-0.266671,0.617893,4.309597,0.269316,0.645497,0.567588,-0.093076,...,-0.271258,-0.473725,0.241280,-0.364842,-0.479485,-0.238216,-0.208425,-0.576184,-0.101094,0.813512,0.220858,-0.324561,0.338646,-0.700138,0.454273,0.513777,1.397234,2.014289,-0.156716,-0.608282,3.313495,-0.416611,0.411518,1.018315,0.531793,-0.144778,-0.091939,0.042629,0.088816,-0.302900,3.888633,0.615814,-0.315690,0,2.580250e+17,1.841013e+15,1.999852e+16,2.186071e+15,2.138800e+17,-9.299200e+13
4,-0.227396,-2.314184,2.399957,0.820347,0.721992,-0.725792,0.802967,0.301240,-0.165655,0.897150,0.583463,0.280083,0.540176,-0.337217,-1.187056,0.858107,-0.163172,0.537501,-0.614731,1.411416,-0.110823,-1.832989,0.434114,0.158721,0.109412,0.864942,-0.153622,-0.406287,1.058695,1.202881,0.355956,2.046729,-0.337899,-0.283286,-0.195394,-0.558610,1.065822,2.127557,-0.367550,-0.081485,...,0.010839,-0.160677,-0.632848,-0.480431,0.168756,-0.239524,0.437287,-0.429804,-0.144387,-0.472897,-0.267123,-0.118421,0.513892,-0.346630,0.925107,-

In [ ]:
import numpy as np

In [ ]:
splits = 5
skf = StratifiedKFold(n_splits=splits, shuffle=True,random_state=42)

oof_preds = np.zeros(X.shape[0])
preds = 0
model_fi = 0
total_mean_auc = 0

for num,(train_idx, valid_idx) in enumerate(skf.split(X,y)):
    X_train, X_valid = X.loc[train_idx], X.loc[valid_idx]
    y_train, y_valid = y.loc[train_idx], y.loc[valid_idx]
    model.fit(X_train,y_train,
              verbose=False,
              eval_set = [(X_train,y_train),(X_valid,y_valid)],
              eval_metric = 'auc',
              early_stopping_rounds = 100,
              )
    preds += model.predict_proba(X_test)[:,1] /splits
    model_fi += model.feature_importances_/splits

    oof_preds[valid_idx] = model.predict_proba(X_valid)[:,1]
    fold_auc = roc_auc_score(y_valid, oof_preds[valid_idx])
    print(f'Fold{num} ROC AUC :{fold_auc}' )
    total_mean_auc += fold_auc / splits

print(f'\nOverall ROC AUC : {total_mean_auc}')

Fold0 ROC AUC :0.809339829756367
Fold1 ROC AUC :0.8109876662183785
Fold2 ROC AUC :0.8096587551517194
Fold3 ROC AUC :0.8099258039376239
Fold4 ROC AUC :0.8088875744629842

Overall ROC AUC : 0.8097599259054146


In [ ]:
importance = pd.DataFrame(model.feature_importances_,index = X.columns, columns=['importance'])
importance = importance.sort_values('importance', ascending = False)
importance

,importance
n_missing,0.803657
f38,0.010054
f34,0.006983
f4,0.004779
f3,0.004584
...,...
f93,0.000000
f94,0.000000
f17,0.000000
f98,0.000000


In [ ]:
submission.claim = preds
submission.head()

,id,claim
0,957919,0.597741
1,957920,0.127836
2,957921,0.594556
3,957922,0.149554
4,957923,0.141015


In [ ]:
submission.to_csv('submission.csv', index = False)